[View in Colaboratory](https://colab.research.google.com/github/ahracho/TIL/blob/master/Fluent_Python/14_Iterator_generator.ipynb)

## 14. 반복형, 반복자, 제너레이터

데이터를 처리할 때 반복은 기본이다. 메모리에 다 들어가지 않는 데이터셋을 검색할 때는 항목들을 `한번에 하나씩, 필요할 때` 가져와야 한다. 

파이썬 3은 제너레이터를 여러 곳에서 사용한다. 예전에는 리스트 전체를 반환했던 내장 함수 range()도 지금은 제너레이터와 비슷한 객체를 반환하기 때문에 range를 사용해서 리스트를 생성해야 한다면, list(range()) 처럼 명시적으로 생성해야 한다.

파이썬의 컬렉션은 모두 반복형이며, 내부적으로 반복자를 사용한다. 

- for loop
- 컬렉션형 생성과 확장
- 텍스트 파일을 한 줄씩 반복
- 지능형 리스트/딕셔너리/집합
- 튜플 언패킹
- 함수 호출 시 \*를 이용해서 실제 매개변수를 언패킹

이번 장에서 다루게 될 내용:
- 반복형 객체를 처리하기 위해 내부적으로 iter() 내장 함수를 사용하는 방법
- 파이썬에서 고전적인 반복자 패턴을 구현하는 방법
- 제너레이터가 작동하는 방식
- 고전적인 반복자를 제너레이터 함수나 제너레이터 표현식으로 바꾸는 방법
- 표준 라이브러리에서의 범용 제너레이터 함수의 활용
- yield from 사용법


### 14.1. Sentence 버전 #1 : 단어 시퀀스


In [0]:
import re
import reprlib

RE_WORD = re.compile("\w+")

class Sentence:
  def __init__(self, text):
    self.text = text
    self.words = RE_WORD.findall(text)
    
  def __getitem__(self, index):
    return self.words[index]
  
  
  def __len__(self):
    return len(self.words)
  
  
  def __repr__(self):
    return "Sentence(%s)" % reprlib.repr(self.text)

In [3]:
s = Sentence('"The time has come," the Walrus said,')
print(s)

for word in s:
  print(word)
  
  
  
print(s[0])
print(s[5])
print(s[-1]) # 시퀀시가 가능한 이유

Sentence('"The time ha... Walrus said,')
The
time
has
come
the
Walrus
said
The
Walrus
said


#### 14.1.1. Sequence가 반복 가능한 이유 : iter()

파이썬 인터프리터가 x 객체를 반복해야 할 때는 언제나 iter(x)를 자동으로 호출한다. 

1. 객체가 __iter__() 메서드를 구현하는지 확인하고, 이를 호출해서 반복자를 가져온다.
2. __iter__() 메서드가 구현되어 있지 않지만 __getitem__()이 구현되어 있다면, 인덱스 0에서 시작해서 항목을 순서대로 가져오는 반복자를 생성한다.
3. 모두 실패하면 TypeError 메시지를 출력하며 에러를 발생시킨다.  


모든 파이썬 시퀀스는 __getitem__()이 구현되어 있기 때문에 반복할 수 있다. 표준 시퀀스는 __iter__() 메서드도 구현하고 있지만, 하위 호환성을 위해 __getitem__()도 같이 구현하는 것이 좋다.  -> 덕타이핑의 극단적인 예

구스 타이핑 기법을 사용하면 __iter__() 특별 메서드를 구현하는 객체만 반복형이라고 간주한다. 

In [4]:
class Foo:
  def __iter__(self):
    pass
  
from collections import abc
print(issubclass(Foo, abc.Iterable))
f = Foo()

print(isinstance(f, abc.Iterable))

print(issubclass(Sentence, abc.Iterable)) # False

True
True
False


## 14.2. 반복형과 반복자
  
**반복형**
> iter() 내장 함수가 반복자를 가져올 수 있는 모든 객체와 반복자를 반환하는 __iter__() 메서드를 구현하는 객체는 반복형이다. 0에서 시작하는 인덱스를 받는 __getitem__() 메서드를 구현하는 객체인 시퀀스도 마찬가지이다. 

반복형 객체에서 반복자를 가져온다. 

In [5]:
s = 'ABC'
for char in s:
  print(char)
  
  
it = iter(s)
while True:
  try:
    print(next(it))
  except StopIteration: # 반복자가 모두 소진되었음을 알려준다
    del it
    break

A
B
C
A
B
C


- __next__() : 다음에 사용할 항목을 반환하고, 더이상 항목이 남아 있지 않으면 StopIteration을 발생시킨다. 
- __iter__() : self를 반환하고, for 루프 등 반복형이 필요한 곳에 반복자를 사용할 수 있게 해준다. 


Iterator ABC 추상 메서드가 it.__next__()이며 이를 직접 호출하면 안되고, next(it) 형태로 호출해야 한다.

In [6]:
s3 = Sentence("Pig and Pepper")
it = iter(s3)
print(it)

print(next(it))
print(next(it))
print(next(it))
# print(next(it)) # error

print(list(it))
print(list(iter(s3))) # 반복자는 재설정 할 수 없고 다시 반복하려면 다시 시작해야한다.

Pig
and
Pepper
[]
['Pig', 'and', 'Pepper']


**반복자** 
> 다음 항목을 반환하거나, 다음 항목이 없을 때 StopIteration 예외를 발생시키는, 인수를 받지 않는 __next__() 메서드를 구현하는 객체. 파이썬 반복자는 __iter__() 메서드도 구현하므로 반복형이기도 하다.


### 14.3. Sentence 버전 #2 : 고전적인 반복자



In [0]:
import re
import reprlib

class Sentence:
  def __init__(self,  text):
    self.text = text
    self.words = RE_WORD.findall(text)
    
  def __repr__(self):
    return 'Sentence(%s)' % reprlib.repr(self.text)
  
  def __iter__(self):
    return SentenceIterator(self.words)
  
  
class SentenceIterator:
  def __init__(self, words):
    self.words = words
    self.index = 0
    
  def __next__(self):
    try:
      word = self.words[self.index]
    except IndexError:
      raise StopIteration()
      
    self.index += 1
    return word
  
  def __iter__(self): # 사실 없어도 돼 하지만 반복자는 next / iter 둘다 있어야 하니까 정직한 방식 -> issubclass 통과
    return self

### 14.3.1. Sentence를 반복자로 만들기 : 좋지 않은 생각

반복형과 반복자를 만드는 데 있어서 흔히 발생하는 오류는 둘을 혼동하기 때문에 발생한다. 반복형은 호출될 때마다 반복자를 새로 생성하는 __iter__()를 가지고 있다. 반복자는 개별 항목을 반환하는 __next__()와 self를 반환하는 __iter__()를 가지고 있다.  

반복자는 반복형이지만, 반복형은 반복자가 아니다.

Sentence 클래스 안에 __iter__()와 __next__()를 구현해서 Sentence 객체를 반복형이자 반복자로 만들고 싶을 수 있지만, 이는 전형적인 안티 패턴이다.  


**반복자 패턴은 다음과 같은 용도로 사용하라.**
- 집합 객체의 내부 표현을 노출시키지 않고 내용에 접근하는 경우
- 집합 객체의 다중 반복을 지원하는 경우
- 다양한 집합 구조체를 반복하기 위한 통일된 인터페이스를 제공하는 경우

다중 반복을 지원하기 위해서는 동일한 반복형 객체로부터 여러 독립적인 반복자를 가질 수 있어야 한다. 그래서 이 패턴을 구현하기 위해서 SentenceIterator 클래스가 필요하다.

### 14.4. Sentence 버전 #3 :제너레이터 함수

동일한 기능을 파이썬스럽게 구현하려면 SequenceIterator 클래스 대신 제너레이터 함수를 사용한다.  

In [0]:
import re
import reprlib

class Sentence:
  def __init__(self, text):
    self.text = text
    self.words = RE_WORD.findall(text)
    
    
  def __repr__(self):
    return "Sentence(%s)" % reprlib.repr(self.text)
  
  def __iter__(self):
    for word in words:
      yield word
    return
  

#### 14.4.1. 제너레이터 함수의 작동 방식

본체 안에 yield 키워드를 가진 함수는 모두 제너레이터 함수이다.  제너레이터 함수는 호출되면 제너레이터 객체를 반환한다. 제너레이터 함수는 제너레이터 팩토리라고 할 수 있다. 

In [9]:
def gen_123():
  yield 1
  yield 2
  yield 3
  
  
print(gen_123) # <function gen_123 at 0x7f224b11e0d0>
print(gen_123()) # 리턴값이 제너레이터 <generator object gen_123 at 0x7f224b100a40>

for i in gen_123():
  print(i)
  
  
g = gen_123()
print(next(g))
print(next(g))
print(next(g))
print(next(g)) # StopIteration

<function gen_123 at 0x7f224b11e0d0>
<generator object gen_123 at 0x7f224b100a40>
1
2
3
1
2
3


StopIteration: ignored

제너레이터 함수는 함수 본체를 포함하는 제너레이터 객체를 생성한다. next()를 제너레이터 객체에 호출하면 함수 본체에 있는 다음 yield로 진행하며, next()는 함수 본체가 중단된 곳에서 생성된 값을 평가한다. 함수 본체가 반환될 때 이 함수를 포함하고 있는 제너레이터 객체는 iterator 프로토콜에 따라 StopIteration 예외를 발생시킨다. 

> **제너레이터는 값을 생성한다.** 함수는 값을 반환하지만, 제너레이터 함수를 호출하면 제너레이터 객체가 반환된다. 제너레이터 객체는 값을 생성한다. 제너레이터 객체는 일반적인 방식으로 값을 반환하지 않는다. 제너레이터 함수 안에 있는 return 문은 제너레이터 객체가 StopIteration 예외를 발생하게 만든다. 

In [11]:
def gen_AB():
  print("Start")
  yield 'A' # gen_AB()에서 처음 next()를 암묵적으로 호출하면 start 출력하고 'A' yield
  print("Continue")
  yield 'B'
  print("end.") # 제너레이터 함수의 끝까지 실행되면 제너레이터 객체는 StopIteration 에러를 발생시키고 for 루프에서 에러를 잡아서 루프를 종료한다.
  
for c in gen_AB(): # for 루프는 g = iter(gen_AB())와 같은 문장을 실행해서 제너레이터 객체를 가져오고, 매번 next(g)를 호출한다.
  print("-->", c)
 

Start
--> A
Continue
--> B
end.


### 14.5. Sentence 버전 #4 : 느긋한 구현

지금까지 구현한 Sentence 클래스는 느긋한 버전이 아니었다. __init__에서 텍스트 안에 있는 단어들의 리스트를 모두 생성하여 words에 바인딩하기 때문이다.  그렇기 때문에 전체 텍스트를 처리해야 하며, 메모리를 많이 차지한다. 

re.finditer() 함수는 re.findall()의 느긋한 버전으로, 리스트 대신 필요에 따라 re.MatchObject 객체를 생성하는 제너레이터를 반환한다. 

In [0]:
import re
import reprlib

class Sentence :
  def __init__(self, text):
    self.text = text
    
  def __repr__(self):
    return "Sentence(%s)" % reprlib.repr(self.text)
  
  def __iter__(self):
    for match in RE_WORD.finditer(self.text): # MatchObject 객체를 생성한다.
      yield match.group()

### 14.6. Sentence 버전 #5 : 제너레이터 표현식

간단한 제너레이터 함수는 제너레이터 표현식으로 바꿀 수 있다. 제너레이터 표현식은 지능형 리스트의 느긋한 버전이라고 할 수 있다. 

제너레이터 표현식은 제너레이터를 생성하고, 제너레이터 표현식을 사용하면 Sentence 클래스의 코드를 더 짧게 만들 수 있다. 

In [15]:
def gen_AB():
  print("start")
  yield "A"
  print("continue")
  yield "B"
  print("end.")
  
  
res1 = [x*3 for x in gen_AB()] # res1은 리스트
print(res1)
for i in res1:
  print("->", i)
  
res2 = (x*3 for x in gen_AB()) # res2는 제너레이터 gen_AB()를 호출하지만, 제너레이터를 여기에서 소비하지 않는다.
print(res2)

for i in res2:
  print("->", i)

start
continue
end.
['AAA', 'BBB']
-> AAA
-> BBB
<generator object <genexpr> at 0x7f224b2a4d00>
start
-> AAA
continue
-> BBB
end.


In [0]:
import re
import reprlib

class Sentence:
  def __init__(self, text):
    self.text = text
    
  def __repr__(self):
    return "Sentence(%s)" % reprlib.repr(self.text)
  
  def __iter__(self):
    return (match.group() for match in RE_WORD.finditer(self.text)) # 제너레이터 표현식으로 부를 때마다 계산해서 생성

### 14.7. 제너레이터 표현식 : 언제 사용하나?

제너레이터 표현식은 함수로 정의하고 호출할 필요 없이 제너레이터를 생성하는 편리 구문이다. 반면 제너레이터 함수는 융통성이 훨씬 더 높아 여러 문장으로 구성된 복잡한 논리를 구현할 수 있고, 심지어 코루틴으로 사용할 수 있다. 제너레이터 표현식이 여러 줄에 걸쳐 있을 때는 가독성을 위해 제너레이터 함수를 사용한다. 게다가 제너레이터 함수는 이름을 가지고 있으므로 재사용할 수도 있다.  

> 제너레이터 표현식을 함수나 생성자에 단일 인수로 전달할 때는 함수를 호출하는 괄호 안에서 제너레이터 표현식을 괄호로 에워쌀 필요가 없다. 


### 14.8. 또 다른 예제 : 등차수열 제너레이터



In [0]:
class ArithmeticProgression:
  def __init__(self, begin, step, end=None):
    self.begin = begin
    self.step = step
    self.end = end
    
  def __iter__(self):
    result = type(self.begin + self.step)(self.begin)
    forever = self.end is None
    index = 0
    while forever or result < self.end:
      yield result
      index += 1
      result = self.begin + self.step*index
    

In [18]:
ap = ArithmeticProgression(0, 1, 3)
print(list(ap))

ap = ArithmeticProgression(1, .5, 3)
print(list(ap))

ap = ArithmeticProgression(1, 1/3, 3)
print(list(ap))

from fractions import Fraction

ap = ArithmeticProgression(0, Fraction(1, 3), 1)
print(list(ap))

from decimal import Decimal
ap = ArithmeticProgression(0, Decimal('.1'), 1)
print(list(ap))

[0, 1, 2]
[1.0, 1.5, 2.0, 2.5]
[1.0, 1.3333333333333333, 1.6666666666666665, 2.0, 2.333333333333333, 2.6666666666666665]
[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]
[Decimal('0'), Decimal('0.1'), Decimal('0.2'), Decimal('0.3'), Decimal('0.4'), Decimal('0.5'), Decimal('0.6'), Decimal('0.7'), Decimal('0.8'), Decimal('0.9')]


In [0]:
# 위의 클래스가 제너레이를 생성하기 위한 목적이었다면 제너레이터 함수로도 구현가능하다.
# 하지만 표준 라이브러리에는 바로 사용할 수 있는 제너레이터가 아주 많기 때문에 굳이?
def aritprog_gen(begin, step, end=None):
  result = type(begin+step)(begin)
  forever = end is None
  index = 0
  while forever or result < end:
    yield result
    index += 1
    result = begin + step*index

#### 14.8.1. itertools 를 사용한 등차수열

itertools에는 다양하게 조합할 수 있는 제너레이터 함수 19개가 있다. 

In [19]:
import itertools

gen = itertools.count(1, .5) # 무한 등차수열 생성 -> list(count()) 실행하면 메모리 폭파
print(next(gen))
print(next(gen))
print(next(gen))


1
1.5


In [20]:
gen = itertools.takewhile(lambda n : n < 3, itertools.count(1, .5)) # 조건이 False가 되면 중단
print(list(gen))

[1, 1.5, 2.0, 2.5]


### 14.9. 표준 라이브러리의 제너레이터 함수

**필터링 제너레이터 함수**

In [24]:
def vowel(c):
  return c.lower() in 'aeiou'

print(list(filter(vowel, "Aardvark")))

import itertools

print(list(itertools.filterfalse(vowel, "Aardvark")))
print(list(itertools.dropwhile(vowel, "Aardvark")))
print(list(itertools.takewhile(vowel, "Aardvark")))
print(list(itertools.compress("Aardvark", (1, 0, 1, 1, 0, 1))))
print(list(itertools.islice("Aardvark", 4)))
print(list(itertools.islice("Aardvark", 4, 7)))
print(list(itertools.islice("Aardvark", 1, 7, 2)))

['A', 'a', 'a']
['r', 'd', 'v', 'r', 'k']
['r', 'd', 'v', 'a', 'r', 'k']
['A', 'a']
['A', 'r', 'd', 'a']
['A', 'a', 'r', 'd']
['v', 'a', 'r']
['a', 'd', 'a']


**매핑 제너레이터 함수**

In [26]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
import itertools

print(list(itertools.accumulate(sample)))
print(list(itertools.accumulate(sample, min)))
print(list(itertools.accumulate(sample, max)))

import operator
print(list(itertools.accumulate(sample, operator.mul)))
print(list(itertools.accumulate(range(1, 11), operator.mul)))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]
[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]
[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]
[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]
[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]


In [27]:
print(list(enumerate("albatroz", 1)))

import operator
print(list(map(operator.mul, range(11), range(11))))

print(list(map(operator.mul, range(11), [2, 4, 8])))

import itertools
print(list(itertools.starmap(operator.mul, enumerate("albatroz", 1))))

print(list(itertools.starmap(lambda a, b : b/a, enumerate(itertools.accumulate(sample), 1))))

[(1, 'a'), (2, 'l'), (3, 'b'), (4, 'a'), (5, 't'), (6, 'r'), (7, 'o'), (8, 'z')]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
[0, 4, 16]
['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']
[5.0, 4.5, 3.6666666666666665, 4.75, 5.2, 5.333333333333333, 5.0, 4.375, 4.888888888888889, 4.5]


**병합 생성자 함수**

In [29]:
print(list(itertools.chain("ABC", range(2))))
print(list(itertools.chain(enumerate("ABC"))))
print(list(itertools.chain.from_iterable(enumerate("ABC"))))

print(list(zip("ABC", range(5))))
print(list(zip("ABC", range(5), [10, 20, 30, 40])))

print(list(itertools.zip_longest("ABC", range(5))))
print(list(itertools.zip_longest("ABC", range(5), fillvalue="?")))

['A', 'B', 'C', 0, 1]
[(0, 'A'), (1, 'B'), (2, 'C')]
[0, 'A', 1, 'B', 2, 'C']
[('A', 0), ('B', 1), ('C', 2)]
[('A', 0, 10), ('B', 1, 20), ('C', 2, 30)]
[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]
[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]


In [30]:
print(list(itertools.product("ABC", range(2))))

suits = "spades hearts diamonds clubs".split()
print(list(itertools.product("AK", suits)))
print(list(itertools.product("ABC")))
print(list(itertools.product("ABC", repeat=2)))
print(list(itertools.product(range(2), repeat=3)))

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]
[('A', 'spades'), ('A', 'hearts'), ('A', 'diamonds'), ('A', 'clubs'), ('K', 'spades'), ('K', 'hearts'), ('K', 'diamonds'), ('K', 'clubs')]
[('A',), ('B',), ('C',)]
[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'B'), ('B', 'C'), ('C', 'A'), ('C', 'B'), ('C', 'C')]
[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]


In [31]:
ct = itertools.count()

print(next(ct))
print(next(ct), next(ct), next(ct))
print(list(itertools.islice(itertools.count(1, .3), 3)))

cy = itertools.cycle("ABC")
print(next(cy))
print(list(itertools.islice(cy, 7)))

rp = itertools.repeat(7)
print(next(rp), next(rp))

print(list(itertools.repeat(8, 4)))
print(list(map(operator.mul, range(11), itertools.repeat(5))))

0
1 2 3
[1, 1.3, 1.6]
A
['B', 'C', 'A', 'B', 'C', 'A', 'B']
7 7
[8, 8, 8, 8]
[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]


In [33]:
print(list(itertools.combinations("ABC", 2)))
print(list(itertools.combinations_with_replacement("ABC", 2)))
print(list(itertools.permutations("ABC", 2)))
print(list(itertools.product("ABC", repeat=2)))

[('A', 'B'), ('A', 'C'), ('B', 'C')]
[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]
[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]
[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'B'), ('B', 'C'), ('C', 'A'), ('C', 'B'), ('C', 'C')]


In [34]:
print(list(itertools.groupby("LLLLAAGGG")))
for char, group in itertools.groupby("LLLLAAGG"):
  print(char, '->', list(group))
  
  
animals = ['duck', 'eagle', 'rat', 'giraffe', 'bear', 'bat', 'dolphin', 'shark', 'lion']
animals.sort(key=len)
print(animals)

for length, group in itertools.groupby(animals, len):
  print(length, '->', list(group))
  
for length, group in itertools.groupby(reversed(animals), len):
  print(length, '->', list(group))

[('L', <itertools._grouper object at 0x7f224b0e4160>), ('A', <itertools._grouper object at 0x7f224b0e4128>), ('G', <itertools._grouper object at 0x7f224b0e4198>)]
L -> ['L', 'L', 'L', 'L']
A -> ['A', 'A']
G -> ['G', 'G']
['rat', 'bat', 'duck', 'bear', 'lion', 'eagle', 'shark', 'giraffe', 'dolphin']
3 -> ['rat', 'bat']
4 -> ['duck', 'bear', 'lion']
5 -> ['eagle', 'shark']
7 -> ['giraffe', 'dolphin']
7 -> ['dolphin', 'giraffe']
5 -> ['shark', 'eagle']
4 -> ['lion', 'bear', 'duck']
3 -> ['bat', 'rat']


In [35]:
print(list(itertools.tee("ABC"))) # 입력된 하나의 반복형에 대한 여러 제너레이터를 생성하고, 각 제너레이터는 독립적으로 항목들을 반복한다.

g1, g2 = itertools.tee("ABC")
next(g1)
next(g2)
next(g2)

print(list(g1))
print(list(g2))

print(list(zip(*itertools.tee("ABC"))))


[<itertools._tee object at 0x7f224b0de6c8>, <itertools._tee object at 0x7f224b0ff8c8>]
['B', 'C']
['C']
[('A', 'A'), ('B', 'B'), ('C', 'C')]


### 14.10. 파이썬 3.3의 새로운 구문 : yield from



In [36]:
def chain(*iterables):
  for it in iterables:
    for i in it:
      yield i
      
      
s = "ABC"
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

In [37]:
'''
yield from 문은 내부 for 루프를 완전히 대체한다. 
'''

def chain(*iterables):
  for i in iterables:
    yield from i
    
    
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

### 14.11. 반복형을 리듀스하는 함수

반복형을 입력받아 하나의 값을 반환하는 함수들을 흔히 '리듀스', '폴딩', '누적'함수라고 한다. 

In [38]:
print(all([1, 2, 3]))
print(all([1, 0, 3]))
print(all([]))
print(any([1, 2, 3]))
print(any([1, 0, 3]))
print(any([0, 0.0]))
print(any([]))

g = (n for n in [0, 0.0, 7, 8])
print(any(g))
print(next(g))

True
False
True
True
True
False
False
True
8


### 14.12. iter() 함수 들여다보기



In [0]:
def d6():
  return randint(1, 6)

d6_iter = iter(d6, 1) # 두번째 인수는 구분 표시로, 콜러블에서 이 값이 반환되면 StopIteration 예외가 발생한다.
print(d6_iter)

for roll in d6_iter:
  print(roll)